In [1]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats

# Parametric test - one way ANOVA
### Analysis of Variance models
The Assumptions of the ANOVA are:
1.	The values in each group come from a normal distribution.
2.	The variances of these normal distributions are equal.
3.	The data are independent within each group.
4.	The data independent between each group.


The goal is to find out if atleast one of the groups have different mean then the others

---

With a p-value less than 0.0001, there is sufficient evidence to conclude that at least two of the judges have different mean of the target variable "pct_women".

# Help:
* [interactions](http://www.statsmodels.org/devel/examples/notebooks/generated/interactions_anova.html)

In [15]:
# test data
data = [
[6.4, "Spock's"],
[8.7, "Spock's"],
[13.3, "Spock's"],
[13.6, "Spock's"],
[15.0, "Spock's"],
[15.2, "Spock's"],
[17.7, "Spock's"],
[18.6, "Spock's"],
[23.1, "Spock's"],
[16.8, "A"],
[30.8, "A"],
[33.6, "A"],
[40.5, "A"],
[48.9, "A"],
[27.0, "B"],
[28.9, "B"],
[32.0, "B"],
[32.7, "B"],
[35.5, "B"],
[45.6, "B"],
[21.0, "C"],
[23.4, "C"],
[27.5, "C"],
[27.5, "C"],
[30.5, "C"],
[31.9, "C"],
[32.5, "C"],
[33.8, "C"],
[33.8, "C"],
[24.3, "D"],
[29.7, "D"],
[17.7, "E"],
[19.7, "E"],
[21.5, "E"],
[27.9, "E"],
[34.8, "E"],
[40.2, "E"],
[16.5, "F"],
[20.7, "F"],
[23.5, "F"],
[26.4, "F"],
[26.7, "F"],
[29.5, "F"],
[29.8, "F"],
[31.9, "F"],
[36.2, "F"]
];
data = pd.DataFrame(data, columns=['pct_women', 'judge'])

# first fit a linear regression
mod = ols('pct_women ~ C(judge)', data=data).fit()

# now do ANOVA
anova_table = sm.stats.anova_lm(mod)
# anova_table is a DataFrame
print('P-value', anova_table['PR(>F)'].iloc[0])
anova_table

P-value 6.09582276794e-05


,sum_sq,df,F,PR(>F)
C(judge),1927.080865,6.0,6.718366,0.000061
Residual,1864.445222,39.0,NaN,NaN


In [24]:
d = data.copy()
d=d[d['judge'].isin(['A', 'B'])]
#d['judge'] = d['judge'] == 'Spock\'s'

mod = ols('pct_women ~ C(judge)', data=d).fit()

# now do ANOVA
anova_table = sm.stats.anova_lm(mod, typ=1)
# anova_table is a DataFrame
print('P-value', anova_table['PR(>F)'].iloc[0])
anova_table

P-value 0.931118241441


/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
C(judge),1.0,0.690939,0.690939,0.007901,0.931118
Residual,9.0,787.056333,87.450704,NaN,NaN


In [9]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

result = pairwise_tukeyhsd(data['pct_women'], data['judge'])
result.summary()

group1,group2,meandiff,lower,upper,reject
A,B,-0.5033,-13.5125,12.5058,False
A,C,-5.02,-17.0032,6.9632,False
A,D,-7.12,-25.0948,10.8548,False
A,E,-7.1533,-20.1625,5.8558,False
A,F,-7.32,-19.3032,4.6632,False
A,Spock's,-19.4978,-31.481,-7.5146,True
B,C,-4.5167,-15.8397,6.8064,False
B,D,-6.6167,-24.1582,10.9249,False
B,E,-6.65,-19.0538,5.7538,False
B,F,-6.8167,-18.1397,4.5064,False


# Nonparametric test - Kruskal-Wallis

In [10]:
a = [63.8, 56.4, 55.2, 58.5, 64.0, 51.6, 54.6, 71.0]
b = [75.5, 83.9, 75.7, 72.5, 56.2, 73.4, 67.7, 87.9]

tstat, pvalue = stats.mstats.kruskalwallis(a, b)
print('test statisic: ', tstat)
print('p-value:', pvalue)

test statisic:  7.45588235294
p-value: 0.00632294769581


# Bonferroni Correction 
we can run t-tests on all pairs, calculate the p-values and apply one of the p-value corrections for multiple testing problems.

The simplest—and at the same time quite conservative—approach is to divide the required p-value by the number of tests that we do (Bonferroni correction). For example, if you perform four comparisons, you check for significance not at p D 0:05, but at p D 0:05=4 D 0:0125.


While multiple testing is not yet included in Python standardly, you can get a number of multiple-testing corrections done with the statsmodels package:


In [27]:
from statsmodels.sandbox.stats.multicomp import multipletests

result = multipletests([.05, 0.3, 0.01], method='bonferroni')
reject, pvals_corrected, alphacSidak, alphacBonf = result
(reject, pvals_corrected, alphacSidak, alphacBonf)

(array([False, False,  True], dtype=bool),
 array([ 0.15,  0.9 ,  0.03]),
 0.016952427508441503,
 0.016666666666666666)

In [11]:
stats.ttest_ind?

In [33]:
from itertools import combinations
from scipy import stats

groups = data['judge'].unique()
pvalues = []
for judge1, judge2 in combinations(groups, 2):
    gr1 = data[data['judge']==judge1]['pct_women']
    gr2 = data[data['judge']==judge2]['pct_women']
    tstat, pvalue = stats.ttest_ind(gr1, gr2, )

    pvalues.append(pvalue)
multipletests(pvalues, method='bonferroni')

(array([ True,  True,  True, False, False,  True, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False], dtype=bool),
 array([  1.97069338e-02,   5.38870338e-04,   1.94941104e-04,
          2.20478320e-01,   9.54741077e-02,   5.30243108e-03,
          1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
          1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
          1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
          1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
          1.00000000e+00,   1.00000000e+00,   1.00000000e+00]),
 0.0024395572596688231,
 0.002380952380952381)

In [39]:
list(combinations(groups, 2))

[("Spock's", 'A'),
 ("Spock's", 'B'),
 ("Spock's", 'C'),
 ("Spock's", 'D'),
 ("Spock's", 'E'),
 ("Spock's", 'F'),
 ('A', 'B'),
 ('A', 'C'),
 ('A', 'D'),
 ('A', 'E'),
 ('A', 'F'),
 ('B', 'C'),
 ('B', 'D'),
 ('B', 'E'),
 ('B', 'F'),
 ('C', 'D'),
 ('C', 'E'),
 ('C', 'F'),
 ('D', 'E'),
 ('D', 'F'),
 ('E', 'F')]

In [96]:
gr1 = data[data['judge']=='B']['pct_women']
gr2 = data[data['judge']=='A']['pct_women']
tstat, pvalue = stats.ttest_ind(gr1, gr2)

print(tstat, pvalue)

-0.0888870152252 0.931118241441


In [83]:
0.931118241441 / 1.029

0.9048768138396502

In [71]:
0.9049

0.9049

In [69]:
stats.f_oneway(data[data['judge']=='Spock\'s']['pct_women'], data[data['judge']=='A']['pct_women'])

F_onewayResult(statistic=18.956195733655658, pvalue=0.00093842542047052331)

In [95]:
multipletests([0.931118241441]*21, method='fdr_tsbky')

(array([False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False], dtype=bool),
 array([ 0.97767415,  0.97767415,  0.97767415,  0.97767415,  0.97767415,
         0.97767415,  0.97767415,  0.97767415,  0.97767415,  0.97767415,
         0.97767415,  0.97767415,  0.97767415,  0.97767415,  0.97767415,
         0.97767415,  0.97767415,  0.97767415,  0.97767415,  0.97767415,
         0.97767415]),
 0.0024395572596688231,
 0.002380952380952381)

In [65]:
(1 - 0.000938425420471) ** 21

0.9804769065388116

In [49]:
import statsmodels.stats as sm_stats

sm_stats.weightstats.ttest_ind?

In [67]:
import statsmodels.stats.multitest as smm
smm.multipletests(.01)

(array([ True], dtype=bool), array([ 0.01]), 0.050000000000000044, 0.05)

In [85]:
multipletests?

In [ ]:
    `bonferroni` : one-step correction
    `sidak` : one-step correction
    `holm-sidak` : step down method using Sidak adjustments
    `holm` : step-down method using Bonferroni adjustments
    `simes-hochberg` : step-up method  (independent)
    `hommel` : closed method based on Simes tests (non-negative)
    `fdr_bh` : Benjamini/Hochberg  (non-negative)
    `fdr_by` : Benjamini/Yekutieli (negative)
    `fdr_tsbh` : two stage fdr correction (non-negative)
    `fdr_tsbky` : two stage fdr correction (non-negative)